In [8]:
import pandas as pd
pd.options.display.max_columns
import pyodbc 
import numpy as np
import os

# Crea Nueva Carpeta (poner nueva)

In [2]:
# Poner nombre de carpeta
Nombrecarpeta = "zzz"
ruta = "C:\\Users\\CerveraM\\San Cristobal S.M.S.G\Marketing - Investigación de mercados - Documentos\\General\\Integracion GSC\\Integración bases\\Bases entrada" 
# Crea carpeta
#os.mkdir(os.path.join(ruta, Nombrecarpeta))

NuevaRuta = os.path.join(ruta, Nombrecarpeta)

# Conexiones

In [3]:
# conecto con SC
conn_sc = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=sqlsrvbi01.database.windows.net;'
                      'Database=sqlsrvdwbi01;'
                      'UID=_Ser_Mkt_SCSMG@sancristobal.com.ar;'
                      'PWD={Lun3s*78*96};'
                      'Authentication=ActiveDirectoryPassword;')

# conecto con Asociart
conn_aso = pyodbc.connect('Driver={SQL Server};'
                      'Server=acen0503.grupoasociart.net;'
                      'Database=DW;'
                      'Trusted_Connection=yes;')

# conecto con Servicios Financieros y Caja Mutual
#conn_sf = pyodbc.connect('Driver={SQL Server};'
#                      'Server=IBM;'
#                      'Database=DW_BBRR;'
#                      'Trusted_Connection=yes;')

# Query y bajada (cambiar fechas en querys)

In [4]:
# Query SC

query_sc ="""
Declare @FechaIni datetime;
Declare @FechaFin datetime;
Set @FechaIni = '20190701'
Set @FechaFin = '20230401'


Select
	   convert(Varchar(4),year(f.FechaEmisionUnificada)) + '-' +  Right('00' + convert(varchar(2),
			 month(f.FechaEmisionUnificada)),2) as AnoMes ,
	   Canal.DescCanal as Canal,
	   Canal.Nivel2 as Canal_N2,
	   Canal.Nivel1 as Canal_N1,
	   DescCasa as Sucursal,
	   Ramo.DescRamo as Ramo,
	   case when SubRamo.DescSubramo = 'SIN DATOS' then  Ramo.DescRamo else SubRamo.DescSubramo end as SubRamo,
	   Prod.CodProductor as PasCod,
	   replace(Prod.NroDocumento, '-' , '') as PasCuit,
	   Prod.NombreProductor as PasNombre,
	   prod.codProductorPrimario as SupraCod,
	   replace(prod.nrodocumentoProductorPrimario, '-' , '') as SupraCuit ,	   
	   prod.NombreCompletoProductorPrimario as SupraNombre,
	   replace(Prod2.NroDocumento, '-' , '') as OrgCuit,
	   Prod.NombreCompletoOrganizador as OrgNombre ,
	   isnull(prod3.SupraCuit , replace(Prod2.NroDocumento, '-' , '') ) as OrgSupraCuit ,
	   isnull(prod3.SupraNombre , Prod.NombreCompletoOrganizador ) as OrgSupraNombre , 
	   replace(Prod4.NroDocumento, '-' , '') as LinkerCuit,
	   Prod.NombreCompletoBroker as LinkerNombre,
	   case when DescCanal = 'AGRARIA' then 1 else 0 end as Flg_Agraria ,
	   case when Prod.NombreCompletoOrganizador = 'San Cristobal' or replace(prod.nrodocumentoProductorPrimario, '-' , '') = prod3.SupraCuit or Prod2.NroDocumento = prod.nrodocumentoProductorPrimario then 0 else 1 end as Flg_org_3ros,
	   prod.NombreEjecutivoOficina as EjecutivoCuenta,
	   Sum(PrimaTransaccion) as PEN,
	   Sum(PrimaTransaccion) * case when Prod.NombreCompletoOrganizador = 'San Cristobal' or replace(prod.nrodocumentoProductorPrimario, '-' , '') = prod3.SupraCuit or Prod2.NroDocumento = prod.nrodocumentoProductorPrimario then 0 else 1 end as PEN_Org
from [dbo].[VW_F_Polizas_Endoso_Vigente] (nolock) f
	   left join D_Productor Prod (nolock) on Prod.CodProductor = f.codProductor
	   left join D_Canal Canal (nolock) on Prod.canalcomercial  = Canal.typecode
	   left join D_Casa Sucursal (nolock)  on Sucursal.CodCasa = Prod.CodCasa
	   left join D_Ramo Ramo on f.codramo=Ramo.ProductCodeRamo
	   left join D_SubRamo SubRamo on f.CodSubRamo=SubRamo.CodSubRamo
	   left join D_Productor Prod2 (nolock) on Prod2.CodProductor = Prod.CodOrganizador
	   left join D_Productor Prod4 (nolock) on Prod4.CodProductor = Prod.CodBroker
	   left join    (
					select PasCuit , SupraCuit , SupraNombre
					from (
						Select
							   Prod.NroDocumento as PasCuit,
							   replace(prod.nrodocumentoProductorPrimario, '-' , '') as SupraCuit ,
							   prod.NombreCompletoProductorPrimario as SupraNombre,
							   row_number() over ( partition by Prod.NroDocumento order by Sum(PrimaTransaccion) desc) as row1
						from [dbo].[VW_F_Polizas_Endoso_Vigente] (nolock) f
							   left join D_Productor Prod (nolock) on Prod.CodProductor = f.codProductor
						where f.FechaEmisionUnificada >= @FechaIni and f.FechaEmisionUnificada < @FechaFin  
						and replace(prod.nrodocumentoProductorPrimario, '-' , '') is not null 
						group by 	   replace(Prod.NroDocumento, '-' , '') ,
							   replace(prod.nrodocumentoProductorPrimario, '-' , '') , Prod.NroDocumento , prod.NombreCompletoProductorPrimario) a
						where row1 = 1 ) Prod3   on Prod2.NroDocumento = Prod3.PasCuit
where f.FechaEmisionUnificada >= @FechaIni and f.FechaEmisionUnificada < @FechaFin 
		--and f.CodSubRamo <> 'AAH_CollectiveMandatoryLife1567' 
		--and DescCanal in ('AGRARIA','BROKERS' ,'TRADICIONAL') and Prod.NroDocumento is not null
group by 	
	   convert(Varchar(4),year(f.FechaEmisionUnificada)) + '-' +  Right('00' + convert(varchar(2),
			 month(f.FechaEmisionUnificada)),2)  ,   Canal.DescCanal ,   Canal.Nivel2 ,	   Canal.Nivel1 ,
	   DescCasa ,   Ramo.DescRamo ,
	   case when SubRamo.DescSubramo = 'SIN DATOS' then  Ramo.DescRamo else SubRamo.DescSubramo end ,
	   replace(Prod.NroDocumento, '-' , '') , Prod.CodProductor ,  Prod.NombreProductor ,  
	   prod.codProductorPrimario  , replace(prod.nrodocumentoProductorPrimario, '-' , '') ,
	   prod.NombreCompletoProductorPrimario ,   replace(Prod2.NroDocumento, '-' , '') ,   Prod.NombreCompletoOrganizador
	 ,   isnull(prod3.SupraCuit , replace(Prod2.NroDocumento, '-' , '') )  ,  isnull(prod3.SupraNombre , Prod.NombreCompletoOrganizador ) , 
	 case when DescCanal = 'AGRARIA' then 1 else 0 end ,
	case when Prod.NombreCompletoOrganizador = 'San Cristobal' or replace(prod.nrodocumentoProductorPrimario, '-' , '') = prod3.SupraCuit then 0 else 1 end ,
	replace(Prod4.NroDocumento, '-' , ''),    Prod.NombreCompletoBroker , prod.NombreEjecutivoOficina """

In [4]:
# Query Asociart

query_aso = """
Declare @FechaIni int;
Declare @FechaFin int;
Set @FechaIni = 202106
Set @FechaFin = 202307

select  	
	left(a1.Mes_Emision_ID,4) + '-' + right(a1.Mes_Emision_ID,2)  as AnoMes,
	a1.Contrato_id as IdContrato,
    a1.Alta_Baja_Id as Es_Alta ,
	a1.Cliente_ID as NroSocio,
	a1.Unidad_Comercial_ID as PasCod,
	a3.Integrante_UC_CUITCUIL as PasCuit,
	a3.Integrante_UC_Desc as PasNombre,
	org.Organizador_CUIT as OrgCuit,
	org.Organizador as OrgNombre,
	a15.Integrante_UC_CUITCUIL as LinkerCuit,
	a15.Integrante_UC_Desc as LinkerNombre,
	a8.Ejecutivo_Cta_Desc as EjecutivoCuenta,  
	a9.Tipo_Canal_Nivel_UC_Desc as Canal,
	a5.Sucursal_Desc as Sucursal,
	a6.Region_Desc as Region,
	a7.SubRegion_Desc as SubRegion,
	'ART' as Ramo,
	a11.Tipo_Contrato_Desc as SubRamo,
	a1.Cto_Prima_Fija + a1.Cto_Prima_Variable as PEN ,
	a1.Cto_Cant_Trabajadores as Cant_Trabajadores ,
	a10.CUIT as Cli_Cuit ,
	a10.Cliente_Desc as Cli_Nombre ,
	a1.ciiu_Nuevo_cto_id as CIIU,
	a13.ciiu_Nuevo_N1_Desc as CIIU_Desc1,
	a14.ciiu_Nuevo_N3_Desc as CIIU_Desc2,
	a12.ciiu_Nuevo_Desc as CIIU_Desc3,
	pen.PEN_anual_A1,
	pen.PEN_anual_A2,
	pen.PEN_anual_A2Ej,
	case when (a1.Mes_Emision_ID = @FechaFin and pen.PEN_anual_A1 > 0) or (a1.Mes_Emision_ID = @FechaFin - 100 and pen.PEN_anual_A2 > 0) 
			or (a1.Mes_Emision_ID = 202306 and pen.PEN_anual_A2Ej > 0)  then 1 else 0 end as Flg_Activo 
from  MSTR_DW.Ft_Contrato_Hist     a1
   join MSTR_DW.Rel_Integrante_UC  a2     on   a1.Unidad_Comercial_ID = a2.Unidad_Comercial_ID
   left join  MSTR_DW.Lk_Integrante_UC a3  on   a2.Integrante_UC_ID=a3.Integrante_UC_ID
   left join MSTR_DW.Lk_Unidad_Comercial a4     on a4.Unidad_Comercial_ID = a1.Unidad_Comercial_ID
   left join MSTR_DW.Lk_Sucursal a5    on a4.Sucursal_UC_Id = a5.Sucursal_Id
   left join MSTR_DW.Lk_Region a6 on a5.Region_Id = a6.Region_Id
   left join MSTR_DW.Lk_SubRegion a7 on a5.SubRegion_Id = a7.SubRegion_Id
   left join MSTR_DW.lk_ejecutivo_cta a8        on a4.Ejec_Cta_uc_Id = a8.Ejecutivo_Cta_Id
   left join MSTR_DW.Lk_Tipo_Canal_Nivel_UC a9 on a2.Tipo_Canal_Nivel_UC_ID = a9.Tipo_Canal_Nivel_UC_ID 
   left join MSTR_DW.Lk_Cliente a10 on a1.Cliente_ID = a10.Cliente_ID	
   left join MSTR_DW.LK_Tipo_Contrato a11 on a11.Tipo_contrato_Id = a1.Tipo_Contrato_Id 
   left join MSTR_DW.Lk_CIIU_Nuevo a12 on a12.ciiu_Nuevo_ID = a1.ciiu_Nuevo_cto_id
   left join MSTR_DW.Lk_CIIU_Nuevo_N1 a13 on a12.ciiu_Nuevo_N1_ID = a13.ciiu_Nuevo_N1_ID 
   left join MSTR_DW.Lk_CIIU_Nuevo_N3 a14 on a12.ciiu_Nuevo_N3_ID = a14.ciiu_Nuevo_N3_ID
   left join MSTR_DW.Lk_Integrante_UC a15 on a4.Nivel_1_UC_ID = a15.Integrante_UC_ID
-- org
   left join   (select a1.Unidad_Comercial_ID , a3.Integrante_UC_Desc as Organizador, Integrante_UC_CUITCUIL as Organizador_CUIT
					from MSTR_DW.Lk_Unidad_Comercial a1
						left join MSTR_DW.Rel_Integrante_UC a2 on a1.Unidad_Comercial_ID = a2.Unidad_Comercial_ID
						left join  MSTR_DW.Lk_Integrante_UC a3  on  a2.Integrante_UC_ID=a3.Integrante_UC_ID
					where a1.Rol_N2_UC_ID = 4 and a2.Nivel_Integrante_ID = 2) org on org.Unidad_Comercial_ID = a1.Unidad_Comercial_ID
-- pen
	left join (select 
					Cliente_ID,
					Sum(case when a1.Mes_Emision_ID >= @FechaFin - 99 and a1.Mes_Emision_ID <=@FechaFin then a1.Cto_Prima_Fija + a1.Cto_Prima_Variable else 0 end ) as PEN_anual_A1,
					Sum(case when a1.Mes_Emision_ID >= @FechaFin - 199 and a1.Mes_Emision_ID <=@FechaFin - 100 then a1.Cto_Prima_Fija + a1.Cto_Prima_Variable else 0 end ) as PEN_anual_A2,
					Sum(case when a1.Mes_Emision_ID >= 202306 - 99 and a1.Mes_Emision_ID <= 202306 then a1.Cto_Prima_Fija + a1.Cto_Prima_Variable else 0 end ) as PEN_anual_A2Ej
				from MSTR_DW.Ft_Contrato_Hist a1
                join MSTR_DW.Rel_Integrante_UC  a2     on   a1.Unidad_Comercial_ID = a2.Unidad_Comercial_ID
				where  a1.Mes_Emision_ID >= @FechaFin - 199 and a1.Mes_Emision_ID <=@FechaFin
					and a2.Ultimo_Nivel_UC_Id in (1)  and a1.Alta_Baja_Id in ('A', 'E')
				group by Cliente_ID ) as pen on a1.Cliente_ID = pen.Cliente_ID
where  a1.Mes_Emision_ID >=  @FechaIni and a1.Mes_Emision_ID <=  @FechaFin
			and a2.Ultimo_Nivel_UC_Id in (1)  and a1.Alta_Baja_Id in ('A', 'E')
"""

In [6]:
# Query SF

query_sf ="""

Declare @FechaIni datetime;
Declare @FechaFin datetime;
Set @FechaIni = '2020-07-01'
Set @FechaFin = '2022-01-31'

select 
	CO.PERIODO,
	CO.COD_SUCURSAL,
	CO.COD_OPERATORIA,
	CO.COD_LINEA,
	CO.NRO_SERVICIO,
	co.FEC_APERTURA FECHA_APERTURA,
	CAST(CONVERT(VARCHAR(6),CO.FEC_APERTURA,112) AS INTEGER) as PERIODO_APERTURA,
	SU.DESC_SUCURSAL AS SUCURSAL,
--	ISNULL(SV.COD_CADENA,0) AS NRO_CADENA
	ISNULL(PER0.NOMBRE_APELLIDO,'SIN VENDEDOR ASIGNADO') AS VENDEDOR_0_NOMBRE ,
	REL0.CUIT as VENDEDOR_0_CUIT,			
	--ISNULL(LOC0.DESC_PROVINCIA,'SIN PROVINCIA') PROVINCIA_VENDEDOR,
	--ISNULL(PER0.PAS_AGENCIA,'') PAS_AGENCIA_PERSONA
	ISNULL(PER1.NOMBRE_APELLIDO,'SIN ZONA') as ZONA,
	ISNULL(PER2.NOMBRE_APELLIDO, 'SIN EJECUTIVO') as EJECUTIVO ,
	ISNULL(PER3.NOMBRE_APELLIDO,'SIN PAS AGENCIA') as PAS_AGENCIA,
	ISNULL(PER4.NOMBRE_APELLIDO,'') as VENDEDOR_4_NOMBRE,
	REL4.CUIT as VENDEDOR_4_CUIT,
	ISNULL(PER5.NOMBRE_APELLIDO,'') AS VENDEDOR_5_NOMBRE,
	REL5.CUIT as VENDEDOR_5_CUIT,			
	ISNULL(PER6.NOMBRE_APELLIDO,'') AS VENDEDOR_6_NOMBRE,
	REL6.CUIT as VENDEDOR_6_CUIT ,
	ISNULL(TITULAR.NOMBRE_APELLIDO,'') NOMBRE_APELLIDO_TITULAR,
--	PM.TASA_LIQUIDACION TASA,
	TITDOC.DESC_TIPO_DOCUMENTO ,
	TITULAR.NRO_DOCUMENTO ,
	case when co.ANULACIONES > 0 then -1 else 1 end as OPERACIONES ,
	case when co.ANULACIONES > 0 then PM.CAPITAL * -1 else PM.CAPITAL end as CAPITAL_PRESTAMO ,
	PM.INTERES_LIQUIDADO INTERES_PRESTAMO
FROM F_RESU_VENTAS_NETAS CO
	INNER JOIN  D_SUCURSAL SU WITH (NOLOCK)  ON SU.COD_SUCURSAL = CO.COD_SUCURSAL
	LEFT JOIN F_SERVICIOS_VENDEDORES SV WITH (NOLOCK) 
					ON SV.COD_SUCURSAL=CO.COD_SUCURSAL
						AND SV.COD_LINEA=CO.COD_LINEA
						AND SV.NRO_SERVICIO=CO.NRO_SERVICIO
   LEFT JOIN CADENA_VENDEDORES_MKT CV  
					ON CV.COD_VENDEDOR_NIVEL_1  = SV.COD_VENDEDOR 	
						AND CV.COD_TIPO_VENDEDOR_NIVEL_1  = SV.COD_TIPO_VENDEDOR
   INNER JOIN F_MAESTROS_OPER MO WITH (NOLOCK)	
					ON MO.COD_SUCURSAL=CO.COD_SUCURSAL
						AND MO.COD_LINEA=CO.COD_LINEA
						AND MO.COD_OPERATORIA=2
						AND MO.NRO_SERVICIO=CO.NRO_SERVICIO
	INNER JOIN F_PRES_MAESTRO PM WITH (NOLOCK)
					ON PM.COD_SUCURSAL=CO.COD_SUCURSAL
						AND PM.COD_LINEA=CO.COD_LINEA
						AND PM.COD_OPERATORIA=2
						AND PM.NRO_SERVICIO=CO.NRO_SERVICIO	
-- Persona 0 VENDEDOR
	LEFT JOIN D_PERSONA_REL REL0 WITH (NOLOCK) 	ON REL0.COD_TIPO_PERSONA =SV.COD_TIPO_VENDEDOR	
													AND REL0.COD_PERSONA = SV.COD_VENDEDOR
	LEFT JOIN D_PERSONAS PER0 WITH (NOLOCK)  ON PER0.COD_TIPO_DOCUMENTO = REL0.COD_TIPO_DOCUMENTO 
													AND PER0.NRO_DOCUMENTO=REL0.NRO_DOCUMENTO
-- Persona 1 ZONA
	LEFT JOIN D_PERSONA_REL REL1 WITH (NOLOCK) ON REL1.COD_TIPO_PERSONA = CV.PER_TIPO_VENDEDOR_REL_1 AND REL1.COD_PERSONA=CV.PER_CODIGO_NIVEL_1	
    LEFT JOIN D_PERSONAS PER1  WITH (NOLOCK) ON PER1.COD_TIPO_DOCUMENTO = REL1.COD_TIPO_DOCUMENTO	AND PER1.NRO_DOCUMENTO=REL1.NRO_DOCUMENTO 			
-- Persona 2 EJECUTIVO CUENTA
	LEFT JOIN D_PERSONA_REL REL2 WITH (NOLOCK) ON REL2.COD_TIPO_PERSONA = CV.PER_TIPO_VENDEDOR_REL_2 AND REL2.COD_PERSONA=CV.PER_CODIGO_NIVEL_2
	LEFT JOIN D_PERSONAS PER2 WITH (NOLOCK) ON PER2.COD_TIPO_DOCUMENTO=	REL2.COD_TIPO_DOCUMENTO	AND PER2.NRO_DOCUMENTO=REL2.NRO_DOCUMENTO 		
-- Persona 3 PAS AGENCIA				
	LEFT JOIN D_PERSONA_REL REL3 WITH (NOLOCK) ON REL3.COD_TIPO_PERSONA=CV.PER_TIPO_VENDEDOR_REL_3 	AND REL3.COD_PERSONA=CV.PER_CODIGO_NIVEL_3
	LEFT JOIN D_PERSONAS PER3 WITH (NOLOCK) ON PER3.COD_TIPO_DOCUMENTO=	REL3.COD_TIPO_DOCUMENTO	AND PER3.NRO_DOCUMENTO=REL3.NRO_DOCUMENTO 	
-- Persona 4
	LEFT JOIN D_PERSONA_REL REL4 WITH (NOLOCK) ON REL4.COD_TIPO_PERSONA=CV.PER_TIPO_VENDEDOR_REL_4 	AND REL4.COD_PERSONA=CV.PER_CODIGO_NIVEL_4
	LEFT JOIN D_PERSONAS PER4 WITH (NOLOCK) ON PER4.COD_TIPO_DOCUMENTO=	REL4.COD_TIPO_DOCUMENTO	AND PER4.NRO_DOCUMENTO=REL4.NRO_DOCUMENTO 	
-- Persona 5	
	LEFT JOIN D_PERSONA_REL REL5 WITH (NOLOCK) ON REL5.COD_TIPO_PERSONA=CV.PER_TIPO_VENDEDOR_REL_5 	AND REL5.COD_PERSONA=CV.PER_CODIGO_NIVEL_5
	LEFT JOIN D_PERSONAS PER5 WITH (NOLOCK) ON PER5.COD_TIPO_DOCUMENTO=	REL5.COD_TIPO_DOCUMENTO	AND PER5.NRO_DOCUMENTO=REL5.NRO_DOCUMENTO 	
-- Persona 6	
	LEFT JOIN D_PERSONA_REL REL6 WITH (NOLOCK) ON REL6.COD_TIPO_PERSONA=CV.PER_TIPO_VENDEDOR_REL_6 AND REL6.COD_PERSONA=CV.PER_CODIGO_NIVEL_6
	LEFT JOIN D_PERSONAS PER6 WITH (NOLOCK) ON PER6.COD_TIPO_DOCUMENTO=	REL6.COD_TIPO_DOCUMENTO	AND PER6.NRO_DOCUMENTO=REL6.NRO_DOCUMENTO 			
-- Titular
	LEFT JOIN D_PERSONA_REL RELTIT  WITH (NOLOCK) ON RELTIT.COD_TIPO_PERSONA=MO.COD_TIPO_PERSONA AND RELTIT.COD_PERSONA=MO.COD_PERSONA
	LEFT JOIN D_PERSONAS TITULAR WITH (NOLOCK) ON TITULAR.COD_TIPO_DOCUMENTO=	RELTIT.COD_TIPO_DOCUMENTO AND TITULAR.NRO_DOCUMENTO=RELTIT.NRO_DOCUMENTO 
	LEFT JOIN D_DOCUMENTO_TIPO TITDOC ON TITDOC.COD_TIPO_DOCUMENTO = RELTIT.COD_TIPO_DOCUMENTO	
where co.FEC_APERTURA >=  dateadd(yy,-1 ,@FechaFin)  and co.FEC_APERTURA < @FechaFin
"""   

In [7]:
# Query SC Clientes

query_sc_clientes ="""
Declare @FechaFin date;
Declare @FechaIni date;
Declare @FechaEj date;
Set @FechaFin = '2023-03-31'
Set @FechaEj = '2022-06-30'
Set @FechaIni = eomonth(dateadd(yyyy , -1, @FechaFin))


select 
	Periodo,
	f.NroPoliza,
	f.NroSocio as Cli_NroSocio,
	replace(isnull(soc.CUIL, soc.CUIT),'-','') as Cli_Cuit ,
	soc.NombreSocio as Cli_Nombre ,
	f.CodProductor as PasCod,
	Prod.NombreProductor as PasNombre,
	replace(Prod.NroDocumento,'-','') as PasCuit,
	replace(prod.nrodocumentoProductorPrimario, '-' , '') as SupraCuit ,	 
	prod.NombreCompletoProductorPrimario as SupraNombre,
	replace(Prod2.NroDocumento, '-' , '') as OrgCuit,
    prod.NombreCompletoOrganizador as OrgNombre,
    Canal.DescCanal as Canal_N3,
	Canal.Nivel2 as Canal_N2,
	Canal.Nivel1 as Canal_N1,
	DescCasa as Sucursal,
	prod.NombreEjecutivoOficina as EjecutivoCuenta,
	prod.CodBroker as LinkerCod,
	prod.NombreCompletoBroker as LinkerNombre,
	Ramo.DescRamo as Ramo,
	case when SubRamo.DescSubramo = 'SIN DATOS' then  Ramo.DescRamo else SubRamo.DescSubramo end as SubRamo,
	f.fechainiciovigencia,
	f.fechafinvigencia,
	soc.PapeleriaDigital,
	soc.RegistradoEnSItioAsegurado as SitioAsegurado,
	soc.RegistradoEnAppMobile as App,
	Prima as PEN
from F_PolizasVigentes f
	left join D_Productor Prod (nolock) on Prod.CodProductor = f.codProductor
	left join D_Productor Prod2 (nolock) on Prod2.CodProductor = Prod.CodOrganizador
	left join D_Canal Canal (nolock) on Prod.canalcomercial  = Canal.typecode
	left join D_Casa Sucursal (nolock)  on Sucursal.CodCasa = Prod.CodCasa
	left join D_Ramo Ramo on f.codramo=Ramo.ProductCodeRamo
	left join D_SubRamo SubRamo on f.CodSubRamo=SubRamo.CodSubRamo
	left join D_Socio Soc on Soc.NroSocio = f.NroSocio
where Periodo in (@FechaIni , @FechaFin, @FechaEj)
 """

In [5]:
# Query SC Autos

query_sc_autos ="""
Declare @FechaFin int;
Declare @FechaIni int;
Declare @FechaEj int;
Set @FechaFin = '202304' 
Set @FechaEj = '202206' 
Set @FechaIni =  @FechaFin - 100


select 
	fecha_cv,
	concat(left(Productor_C,2),'-',right(Productor_C,6)) as PasCod,
	replace(Prod.NroDocumento, '-' , '') as PasCuit,
	Prod.NombreProductor as PasNombre,
	replace(prod.nrodocumentoProductorPrimario, '-' , '') as SupraCuit ,
	prod.NombreCompletoProductorPrimario as SupraNombre,
	count(*) as Autos,
	Canal_D as Canal,
	Suc_D as Sucursal,
	categoria_Agrup_D as Categoria
from F_RiesgosVigentes_Auto
	left join D_Productor Prod (nolock) on Prod.CodProductor = concat(left( Productor_C ,2),'-',right( Productor_C ,6))
where fecha_cv in (@FechaIni , @FechaFin, @FechaEj)
group by fecha_cv, concat(left(Productor_C,2),'-',  right(Productor_C,6)) ,  Suc_D,   Canal_D,  prod.CodOrganizador ,
	prod.NombreCompletoOrganizador , categoria_Agrup_D,    replace(Prod.NroDocumento, '-' , '') ,
	   Prod.NombreProductor ,	   replace(prod.nrodocumentoProductorPrimario, '-' , '')  ,
	   prod.NombreCompletoProductorPrimario ,	   	   Prod.NombreCompletoOrganizador

 """

In [12]:
pas_sc = pd.read_sql_query(query_sc, conn_sc)
cli_sc = pd.read_sql_query(query_sc_clientes, conn_sc)
auto_sc = pd.read_sql_query(query_sc_autos, conn_sc)
#pas_sf = pd.read_sql_query(query_sf, conn_sf)

In [6]:
auto_sc = pd.read_sql_query(query_sc_autos, conn_sc)

C:\Users\tamuchs\AppData\Local\Temp\ipykernel_23268\1747119954.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  auto_sc = pd.read_sql_query(query_sc_autos, conn_sc)


In [8]:
pas_aso = pd.read_sql_query(query_aso, conn_aso)
pas_aso.to_csv(os.path.join(NuevaRuta, "dw_Aso.csv"), index = False)

In [12]:
pas_sc.to_csv(os.path.join(NuevaRuta, "dw_SC.csv"), index = False)
pas_aso.to_csv(os.path.join(NuevaRuta, "dw_Aso.csv"), index = False)
cli_sc.to_csv(os.path.join(NuevaRuta, "dw_SC_cli.csv"), index = False)
auto_sc.to_csv(os.path.join(NuevaRuta, "dw_SC_auto.csv"), index = False)
#pas_sf.to_csv(os.path.join(NuevaRuta, "dw_SF.csv"), index = False)

In [7]:
auto_sc.to_csv(os.path.join(NuevaRuta, "dw_SC_auto.csv"), index = False)

OSError: Cannot save file into a non-existent directory: 'C:\Users\CerveraM\San Cristobal S.M.S.G\Marketing - Investigación de mercados - Documentos\General\Integracion GSC\Integración bases\Bases entrada\zzz'